In [ ]:
'''BufferErrorimport pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE'''

In [ ]:
# Load data

'''df = pd.read_csv('UTI.csv')
print(df)
# Convert 'yes'/'no' to 1/0 for all categorical features
binary_columns = ['Occurrence of nausea', 'Lumbar pain', 'Urine pushing (continuous need for urination)', 
                  'Micturition pains', 'Burning of urethra, itch, swelling of urethra outlet', 
                  'Inflammation of urinary bladder', 'Nephritis of renal pelvis origin']

for col in binary_columns:
    df[col] = df[col].map({'yes': 1, 'no': 0})

# Define features and target
X = df.drop('Nephritis of renal pelvis origin', axis=1)
y = df['Nephritis of renal pelvis origin']'''

     Temperature of patient Occurrence of nausea Lumbar pain  \
0                      35.5                   no         yes   
1                      35.9                   no          no   
2                      35.9                   no         yes   
3                      36.0                   no          no   
4                      36.0                   no         yes   
..                      ...                  ...         ...   
115                    41.4                   no         yes   
116                    41.5                   no          no   
117                    41.5                  yes         yes   
118                    41.5                   no         yes   
119                    41.5                   no         yes   

    Urine pushing (continuous need for urination) Micturition pains  \
0                                              no                no   
1                                             yes               yes   
2                 

In [ ]:
# Stratified split to preserve class distribution
'''X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)'''

NameError: name 'X' is not defined

In [ ]:

'''pipeline = Pipeline([
    ('scaler', StandardScaler()),       # Scale features
    ('smote', SMOTE(random_state=42)),  # Balance classes via oversampling -Synthetic Minority Over Sampling Technique
    ('svm', SVC(probability=True, random_state=42))  # Enable probability
])'''

In [ ]:
# Define parameter grid
'''param_grid = {
    'svm__C': [0.1, 1, 10],                # Regularization strength
    'svm__kernel': ['linear', 'rbf'],  #radial basis function     # Kernel type
    'svm__gamma': ['scale', 'auto', 0.1]    # Kernel coefficient (for RBF)
}

# Use stratified 5-fold cross-validation
stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Run grid search
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=stratified_cv,
    scoring='roc_auc',  # Prioritize AUC for imbalanced data-reciever operating characteristic
    n_jobs=-1 
)
grid_search.fit(X_train, y_train)'''

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('smote', SMOTE(random_state=42)),
                                       ('svm',
                                        SVC(probability=True,
                                            random_state=42))]),
             n_jobs=-1,
             param_grid={'svm__C': [0.1, 1, 10],
                         'svm__gamma': ['scale', 'auto', 0.1],
                         'svm__kernel': ['linear', 'rbf']},
             scoring='roc_auc')

In [ ]:
# Get best model
'''best_model = grid_search.best_estimator_

# Predict on test data
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# Print metrics
print("Best Parameters:", grid_search.best_params_)
print("Accuracy:", round(accuracy_score(y_test, y_pred), 4))

print("\nClassification Report:\n", classification_report(y_test, y_pred))'''

Best Parameters: {'svm__C': 0.1, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}
Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        10

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24



In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [4]:
# Load data
df = pd.read_csv('UTI.csv')
print(df)
# Convert 'yes'/'no' to 1/0 for all categorical features
binary_columns = ['Occurrence of nausea', 'Lumbar pain', 'Urine pushing (continuous need for urination)', 
                  'Micturition pains', 'Burning of urethra, itch, swelling of urethra outlet', 
                  'Inflammation of urinary bladder', 'Nephritis of renal pelvis origin']

for col in binary_columns:
    df[col] = df[col].map({'yes': 1, 'no': 0})

# Define features and target
X = df.drop('Nephritis of renal pelvis origin', axis=1)
y = df['Nephritis of renal pelvis origin']


# Stratified split to preserve class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

     Temperature of patient Occurrence of nausea Lumbar pain  \
0                      35.5                   no         yes   
1                      35.9                   no          no   
2                      35.9                   no         yes   
3                      36.0                   no          no   
4                      36.0                   no         yes   
..                      ...                  ...         ...   
115                    41.4                   no         yes   
116                    41.5                   no          no   
117                    41.5                  yes         yes   
118                    41.5                   no         yes   
119                    41.5                   no         yes   

    Urine pushing (continuous need for urination) Micturition pains  \
0                                              no                no   
1                                             yes               yes   
2                 

In [ ]:
# Calculate class weights for CatBoost
scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)

In [6]:
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.05,
    depth=5,
    l2_leaf_reg=3,
    scale_pos_weight=scale_pos_weight,  # Critical for imbalance
    eval_metric='AUC',
    early_stopping_rounds=50,
    random_seed=42,
    verbose=0  # Disable training logs
)

In [7]:
# Use stratified 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [8]:
# Final training on full data
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Metrics
print("Test Accuracy:", round(accuracy_score(y_test, y_pred), 4))
print("Test ROC AUC:", round(roc_auc_score(y_test, y_proba), 4))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Test Accuracy: 1.0
Test ROC AUC: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        10

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24



In [ ]:
'''import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder'''

In [ ]:
# Load data
'''df = pd.read_csv('UTI.csv')
print(df)
# Convert 'yes'/'no' to 1/0 for all categorical features
binary_columns = ['Occurrence of nausea', 'Lumbar pain', 'Urine pushing (continuous need for urination)', 
                  'Micturition pains', 'Burning of urethra, itch, swelling of urethra outlet', 
                  'Inflammation of urinary bladder', 'Nephritis of renal pelvis origin']

for col in binary_columns:
    df[col] = df[col].map({'yes': 1, 'no': 0})

# Define features and target
X = df.drop('Nephritis of renal pelvis origin', axis=1)
y = df['Nephritis of renal pelvis origin']'''

     Temperature of patient Occurrence of nausea Lumbar pain  \
0                      35.5                   no         yes   
1                      35.9                   no          no   
2                      35.9                   no         yes   
3                      36.0                   no          no   
4                      36.0                   no         yes   
..                      ...                  ...         ...   
115                    41.4                   no         yes   
116                    41.5                   no          no   
117                    41.5                  yes         yes   
118                    41.5                   no         yes   
119                    41.5                   no         yes   

    Urine pushing (continuous need for urination) Micturition pains  \
0                                              no                no   
1                                             yes               yes   
2                 

In [ ]:
# Calculate class weight ratio for XGBoost --to balance
#scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)

In [ ]:
# Define parameter grid
'''param_grid = {
    'max_depth': [3, 5, 7],           # Tree depth
    'learning_rate': [0.01, 0.05, 0.1],  # Step size shrinkage
    'n_estimators': [100, 200],       # Number of trees
    'subsample': [0.7, 0.9]           # Fraction of samples used per tree
}

# Initialize XGBoost model
model = XGBClassifier(
    objective='binary:logistic',
    scale_pos_weight=scale_pos_weight,
    random_state=42
)

# Stratified 5-fold cross-validation
stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Run grid search
grid_search = GridSearchCV(
    model,
    param_grid,
    cv=stratified_cv,
    scoring='roc_auc',  # Prioritize AUC for imbalanced data
    #roc=reciever operating characteristic
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

# Get best model
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)'''

Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}


In [ ]:
# Predictions
'''y_pred = best_model.predict(X_test)
print(y_pred)
y_proba = best_model.predict_proba(X_test)[:, 1]
print(y_proba)

# Metrics
print("Test Accuracy:", round(accuracy_score(y_test, y_pred), 4))
print("\nClassification Report:\n", classification_report(y_test, y_pred))'''

[0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0 1 1 0 1]
[0.01506347 0.04695912 0.11924237 0.9797061  0.9802201  0.9769645
 0.9797061  0.04695912 0.01506347 0.94631016 0.01506347 0.93957275
 0.11364786 0.01506347 0.01506347 0.04695912 0.93957275 0.01506347
 0.04695912 0.11364786 0.9467682  0.97918504 0.01506347 0.9715206 ]
Test Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        10

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24



In [10]:
import joblib

# Save the model to a pickle file
joblib.dump(model, 'catboost_uti_model.pkl')
print("Model saved to catboost_uti_model.pkl")


Model saved to catboost_uti_model.pkl


In [ ]:
# Function to Predict UTI/Nephritis for a new patient
'''def predict_uti():
    print("\nEnter patient details:")

    temp = float(input("Temperature of patient (°C): "))
    nausea = int(input("Occurrence of nausea (1: Yes, 0: No): "))
    lumbar_pain = int(input("Lumbar pain (1: Yes, 0: No): "))
    urine_pushing = int(input("Continuous need for urination (1: Yes, 0: No): "))
    micturition_pain = int(input("Micturition pain (1: Yes, 0: No): "))
    urethra_issues = int(input("Urethra burning/itch/swelling (1: Yes, 0: No): "))
    bladder_inflammation = int(input("Inflammation of urinary bladder (1: Yes, 0: No): "))

    # Create input for prediction
    patient_data = np.array([[temp, nausea, lumbar_pain, urine_pushing,
                              micturition_pain, urethra_issues, bladder_inflammation]])
    
    # Make prediction
    prediction = grid_search.predict(patient_data)

    if prediction[0] == 1:
        print("\n Diagnosis: The patient may have UTI.")
    else:
        print("\n Diagnosis: No signs of UTI.")

# Predict on a new patient
predict_uti()'''


Enter patient details:

 Diagnosis: No signs of UTI.
